<a href="https://colab.research.google.com/github/felixsimard/comp551-p4/blob/main/results_reproduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!rm -rf apex
!rm -rf fairseq

Install required dependencies

In [1]:
!pip install tensorboardX

     |████████████████████████████████| 124 kB 4.0 MB/s 


In [2]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 30087, done.
remote: Counting objects: 100% (1244/1244), done.
remote: Compressing objects: 100% (742/742), done.
remote: Total 30087 (delta 591), reused 939 (delta 482), pack-reused 28843
Receiving objects: 100% (30087/30087), 14.28 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (22266/22266), done.
/content/fairseq
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 123 kB 4.1 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 88 kB 7.4 MB/s 
     |████████████████████████████████| 112 kB 42.0 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=b7d87b37c9c8f919a6ea9

In [3]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./
%cd ..

Cloning into 'apex'...
remote: Enumerating objects: 8639, done.
remote: Counting objects: 100% (1109/1109), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 8639 (delta 1000), reused 895 (delta 895), pack-reused 7530
Receiving objects: 100% (8639/8639), 14.33 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (5902/5902), done.
/content/apex
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/

Preprocess GLUE data

In [4]:
%cd fairseq
!./examples/roberta/preprocess_GLUE_tasks.sh /content/drive/MyDrive/glue_data MNLI
%cd ..

/content/fairseq
--2021-12-08 17:29:30--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘encoder.json’

encoder.json        100%[===================>]   1018K  1.08MB/s    in 0.9s    

2021-12-08 17:29:32 (1.08 MB/s) - ‘encoder.json’ saved [1042301/1042301]

--2021-12-08 17:29:32--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================

Finetune RoBERTa

In [6]:
%cd fairseq

!TOTAL_NUM_UPDATES=2036
!WARMUP_UPDATES=122
!LR=2e-05
!NUM_CLASSES=2
!MAX_SENTENCES=16
!ROBERTA_PATH=/content/drive/MyDrive/roberta.large/model.pt

!fairseq-train /content/fairseq/MNLI-bin \
    --restore-file /content/drive/MyDrive/roberta.large/model.pt \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch roberta_large \
    --criterion sentence_prediction \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 2e-05 --total-num-update 2036 --warmup-updates 122 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --find-unused-parameters \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --ddp-backend legacy_ddp \
    --quant-noise-pq 0.2 --quant-noise-pq-block-size 8

%cd ..


/content/fairseq
2021-12-08 17:58:37 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_pro

In [11]:
!CUDA_VISIBLE_DEVICES=0 fairseq-hydra-train --config-dir /content/fairseq/examples/roberta/config/finetuning --config-name mnli \
task.data=/content/fairseq/MNLI-bin checkpoint.restore_file=$ROBERTA_PATH

[2021-12-08 18:03:55,402][fairseq_cli.train][INFO] - {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 200, 'log_format': 'json', 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 4, 'fp16_scale_window': 128, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': 1.0, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_name': None, 'path': None, 'post_process': None